In [79]:
import pandas as pd
import numpy as np
import torch
import cv2
import PIL
from PIL import Image
import torchvision

import math
import os
from pathlib import Path
from random import randrange

import timm

In [31]:
base_path = "./data_split/"

Path(base_path + "train").mkdir(parents=True, exist_ok=True)
Path(base_path + "val").mkdir(parents=True, exist_ok=True)
Path(base_path + "test").mkdir(parents=True, exist_ok=True)

In [7]:
def get_file_name(base):
    for path, dir, file in os.walk(base):
        for f in file:
            if ".png" in f:
                yield(os.path.join(path, f))

In [8]:
fake_type = []
for idx, file_path in enumerate(get_file_name('./data/manipulated/')):
    fake_type.append(((file_path.split('/'))[-1]).split('_')[0])
    
fake_type = np.array(fake_type)
fake_type = np.unique(fake_type).tolist()
print(fake_type)

['DF', 'F2F', 'FS', 'NT', 'eyes', 'mouth']


In [19]:
image_data = {
    'real': [],
    'DF': [],
    'F2F': [],
    'FS': [],
    'NT': [],
    'eyes': [],
    'mouth': []
}

for idx, file_path in enumerate(get_file_name('./data/')):
    img = Image.open(file_path)
    
    if 'manipulated' in file_path: 
        for i, t in enumerate(fake_type):
            if t in file_path:
                image_data[t].append(img)
                break
    else:
        image_data['real'].append(img)
        
    img.close()

In [29]:
for k, v in image_data.items():
    print(k, len(v), v[0])

real 4000 ./data/original/3975.png
DF 1333 ./data/manipulated/DF_745.png
F2F 1335 ./data/manipulated/F2F_568.png
FS 1333 ./data/manipulated/FS_474.png
NT 1333 ./data/manipulated/NT_921.png
eyes 1333 ./data/manipulated/eyes_633.png
mouth 1333 ./data/manipulated/mouth_1314.png


In [32]:
train = 0.7
val = 0.15
test = 0.15

train_path = "./data_split/train/"
val_path = "./data_split/val/"
test_path = "./data_split/test/"

for k, v in image_data.items():
    train_len = math.ceil(len(v)*train)
    val_len = math.floor(len(v)*val)
    test_len = len(v) - train_len - val_len
    print(k, train_len, val_len, test_len)
    
    for idx, t in enumerate(v):
        t = Image.open(t.filename)
        if idx <= train_len:
            t.save(train_path + t.filename.split("/")[-1],"PNG")
        elif idx > train_len and idx <= (train_len+val_len):
            t.save(val_path + t.filename.split("/")[-1],"PNG")
        else:
            t.save(test_path + t.filename.split("/")[-1],"PNG")
        t.close()

real 2800 600 600
DF 934 199 200
F2F 935 200 200
FS 934 199 200
NT 934 199 200
eyes 934 199 200
mouth 934 199 200


In [45]:
# train 9600 1067

In [81]:
for idx, file_path in enumerate(get_file_name(train_path)):
    if "_" in file_path.split("/")[-1]:
        img = Image.open(file_path)
        rand = randrange(4)
        
        if rand == 0:
            img_m = torchvision.transforms.RandomRotation((10, 20))(img)
            img_m.save(img.filename[:-4]+"-rot.png", "PNG")
        elif rand == 1:
            img_m = torchvision.transforms.RandomHorizontalFlip()(img)
            img_m.save(img.filename[:-4]+"-hf.png", "PNG")
        elif rand == 2:
            img_m = torchvision.transforms.RandomRotation((10, 20))(img)
            img_m.save(img.filename[:-4]+"-rot.png", "PNG")
            img_m = torchvision.transforms.RandomHorizontalFlip()(img)
            img_m.save(img.filename[:-4]+"-hf.png", "PNG")
        elif rand == 3:
            img_m = torchvision.transforms.RandomHorizontalFlip()(img)
            img_m = torchvision.transforms.RandomRotation((10, 20))(img_m)
            img_m.save(img.filename[:-4]+"-hfrot.png", "PNG")

In [82]:
count = 0
for idx, file_path in enumerate(get_file_name(train_path)):
    if "-" in file_path.split("/")[-1]: count += 1
count

7417

In [175]:
9600 + 7988

17588

In [176]:
17588 // 32

549

In [179]:
32 * 600

19200

In [182]:
torchvision.models.resnet50(pretrained=True)

AttributeError: module 'torchvision.models' has no attribute 'ResNet50_Weights'